In [7]:
from util import *
import time
import datetime

import cPickle as pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pylab
import re
import scipy as sp
import seaborn
import sklearn.feature_extraction.text as text

from gensim import corpora, models
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import decomposition

lines=[]
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  
    
userid = []
rating = []
docs = []
business = []
i = 0
j = 0
error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        docs.append(l[3])
        #d = clean(l[3])
        #kmers = getKmers(d)
        #d.extend(kmers)
        #docs.append(d)
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)

Training Data: Number of lines processed: 317586
Training Data: Length of userid array: 317586
Training Data: Length of rating array: 317586
Training Data: Length of docs array: 317586
Training Data: Length of business array: 317586
Training Data: Number of exceptions encountered: 0


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsRegressor

total_data =50000
train_data =45000

def getSentiment(x):
    if x < 3.5:
        return 0
    else:
        return 1
    

def nmf(rating):
    nmf = load_pickle('nmf.pickle')
    print nmf.shape
    trainTopics= nmf[:total_data]
    rating=rating[:total_data]
    print trainTopics.shape
    print len(rating)
    trainTopics = trainTopics / np.sum(trainTopics, axis=1, keepdims=True)
    d, f = trainTopics.shape
    cols = ["Topic"+str(i) for i in xrange(1, f+1)]
    nmfDF = pd.DataFrame(trainTopics, columns=cols)
    nmfDF['rating'] = map(float,rating)
    cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20']
    Xtrain = nmfDF[:train_data][cols]
    Ytrain = nmfDF[:train_data]['rating']
    Xtest = nmfDF[train_data:][cols]
    Ytest = nmfDF[train_data:]['rating']
    
    return nmfDF, Xtrain, Ytrain, Xtest, Ytest
    
    
def nmf_sentiment(rating):
    nmfDF,a,b,c,d = nmf(rating)
    nmfDF['Sentiment'] = nmfDF['rating'].map(getSentiment)
    nmfDF = nmfDF.dropna(how='any')
    cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20', u'Sentiment']
    Xtrain = nmfDF[:train_data][cols]
    Ytrain = nmfDF[:train_data]['rating']
    Xtest = nmfDF[train_data:][cols]
    Ytest = nmfDF[train_data:]['rating']
    
    return nmfDF,Xtrain,Ytrain,Xtest,Ytest

    
    
def lda_sentiment(rating):
    lda = load_pickle('lda.pickle')
    trainTopics= lda[:total_data]
    rating=rating[:total_data]
    trainTopics = trainTopics / np.sum(trainTopics, axis=1, keepdims=True)
    d, f = trainTopics.shape
    cols = ["Topic"+str(i) for i in xrange(1, f+1)]
    ldaDF = pd.DataFrame(trainTopics, columns=cols)
    ldaDF['rating'] = map(float,rating)
    ldaDF['Sentiment'] = ldaDF['rating'].map(getSentiment)
    ldaDF = ldaDF.dropna(how='any')
    cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20', u'Sentiment']
    Xtrain = ldaDF[:train_data][cols]
    Ytrain = ldaDF[:train_data]['rating']
    Xtest = ldaDF[train_data:][cols]
    Ytest = ldaDF[train_data:]['rating']
    
    return ldaDF,Xtrain,Ytrain,Xtest,Ytest
    


In [43]:
rating = map(float,rating)
print len(rating)

nmdf,m,n,o,p = nmf(rating)
nmsdf,q,w,e,r = nmf_sentiment(rating)

ldf,a,b,c,d = lda_sentiment(rating)



317586
(50000, 20)
(50000, 20)
50000
(50000, 20)
(50000, 20)
50000


In [44]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge


from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score



ModelEvaluationResults = {}

rating = map(float,rating)

models= [nmf(rating),nmf_sentiment(rating),lda_sentiment(rating)]
model_names = ["NMF","NMF-Sentiment","LDA-Sentiment"]


results = {}

for (m,models_) in enumerate(models):
    df,train_X,train_Y,test_X,test_Y = models_
    print "Evalutaion of Model:  "+str(model_names[m])
    
    clfs = [ LogisticRegression(),
    KNeighborsRegressor(n_neighbors=3),
    SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
    ]
    #KernelRidge(alpha=1.0, coef0=1, degree=3, gamma=None, kernel='linear',kernel_params=None)]

    #clf_names = ['Logistic Regression','KNeighborsRegressor','SVR','KernelRidge']
    clf_names = ['Logistic Regression','KNeighborsRegressor','SVR']


    
    ClassifierEvaluationResults = {}
    
    for (i, clf_) in enumerate(clfs):
        #clf = clf_.fit(Xtrain, Ytrain)
        #preds = clf_.predict(Xtest)
        print "********************************************************************"
        print "Training validations evaluations for : "+str(clf_names[i])
        scores = cross_val_score(clf_, train_X, train_Y, cv=5)
        predicted_ratings = cross_val_predict(clf_, test_X,test_Y, cv=5)
        print scores
        train_mean_score=scores.mean()
        print "Testing validations evaluations for : "+str(clf_names[i])


        r2 = r2_score(test_Y, predicted_ratings, multioutput='uniform_average')
        mae = median_absolute_error(test_Y, predicted_ratings)
        msle =mean_squared_log_error(test_Y, predicted_ratings) 
        mse = mean_squared_error(test_Y, predicted_ratings)
        mae = mean_absolute_error(test_Y, predicted_ratings)
        evs = explained_variance_score(test_Y, predicted_ratings)  
        
        data ={'training score':train_mean_score,
            'r2 score':r2,
            'median absolute error':mae,
            'mean squared log error':msle,
            'mean sqaured error':mse,
            'mean absolute error':mae,
            'explained variance error':evs}

        #data ={'training_score':train_mean_score}        
        
        ClassifierEvaluationResults[clf_names[i]] = data
        print data
    
    ModelEvaluationResults[model_names[m]]=ClassifierEvaluationResults
    print ClassifierEvaluationResults
    print "********************************************************************"




(50000, 20)
(50000, 20)
50000
(50000, 20)
(50000, 20)
50000
Evalutaion of Model:  NMF
********************************************************************
Training validations evaluations for : Logistic Regression
[ 0.45512108  0.45317187  0.43655556  0.45182798  0.45087797]
Testing validations evaluations for : Logistic Regression
{'r2 score': 0.007057749106524791, 'mean absolute error': 0.70799999999999996, 'mean sqaured error': 1.0915999999999999, 'training score': 0.44951089266418781, 'median absolute error': 0.70799999999999996, 'explained variance error': 0.084403329576598907, 'mean squared log error': 0.063683733764986664}
********************************************************************
Training validations evaluations for : KNeighborsRegressor
[ 0.04514013  0.00296003  0.05560693  0.01632206  0.01310399]
Testing validations evaluations for : KNeighborsRegressor
{'r2 score': 0.024178837142573295, 'mean absolute error': 0.80526666666666669, 'mean sqaured error': 1.07277777777

In [56]:


cols = ['training score', 'r2 score', 'median absolute error', 'mean squared log error','mean sqaured error','mean absolute error','explained variance error']
#cols = ['training score', 'r2 score', 'median absolute error', 'mean squared log error','mean sqaured error','mean absolute error','explained variance error']
clf_names = ['Logistic Regression','KNeighborsRegressor','SVR']
model_names = ["NMF","NMF-Sentiment","LDA-Sentiment"]


#print ModelEvaluationResults
#cols = ['precision', 'recall', 'f1_score', 'accuracy']
#pd.DataFrame(SentimentResults).T[cols].T

#temp = ModelEvaluationResults["NMF"]['Logistic Regression']
#temp.T


#print pd.DataFrame(temp).T

#for m in model_names:
    #print ModelEvaluationResults[m].T[clf_names]
    #print pd.DataFrame(m)
    #print pd.DataFrame(m).T[clf_names].T
    #temp = ModelEvaluationResults[m]
    #print pd.DataFrame(temp).T[cols].T

In [45]:
ModelEvaluationResults

{'LDA-Sentiment': {'KNeighborsRegressor': {'explained variance error': 0.64479244925654156,
   'mean absolute error': 0.50473333333333326,
   'mean sqaured error': 0.39082222222222224,
   'mean squared log error': 0.025886269597315936,
   'median absolute error': 0.50473333333333326,
   'r2 score': 0.64449991110981741,
   'training score': 0.64140528731690993},
  'Logistic Regression': {'explained variance error': 0.54552731182443592,
   'mean absolute error': 0.43440000000000001,
   'mean sqaured error': 0.50160000000000005,
   'mean squared log error': 0.032528844781925276,
   'median absolute error': 0.43440000000000001,
   'r2 score': 0.54373412142894173,
   'training score': 0.61131090147187073},
  'SVR': {'explained variance error': 0.66749928886344545,
   'mean absolute error': 0.47844083185340908,
   'mean sqaured error': 0.36934001368078118,
   'mean squared log error': 0.025507099773621322,
   'median absolute error': 0.47844083185340908,
   'r2 score': 0.6640405784818415,
  